In [2]:
using ITensors

In [8]:
using Plots

In [5]:
function Calc_SvN(psi::MPS, b::Int64)
    # Orthogonalize the MPS at bond index b
    orthogonalize!(psi, b)

    # Perform Singular Value Decomposition (SVD)
    U, S, V = svd(psi[b], (linkind(psi, b-1), siteind(psi, b)))

    # Calculate von Neumann entropy
    SvN = 0.0
    for n = 1:dim(S, 1)
        p = S[n, n]^2
        SvN -= p * log(p)
    end

    return SvN
end;

In [6]:
function TFIM_DMRG(N::Int64, h::Float64, swp_num::Int64, maxM::Int64)
"""
    TFIM_DMRG(N::Int64, h::Float64, swp_num::Int64, maxM::Int64)

    Perform a Density Matrix Renormalization Group (DMRG) simulation for the Transverse Field Ising Model (TFIM) on a 1D chain.

    Parameters:
    - `N::Int64`: Number of sites in the 1D quantum system.
    - `h::Float64`: Transverse field strength in the TFIM Hamiltonian.
    - `swp_num::Int64`: Number of DMRG sweeps.
    - `maxM::Int64`: Maximum bond dimension.

    Returns:
    - `energy::Float64`: Ground state energy obtained from the DMRG simulation.
    - `psi::MPS`: Matrix Product State (MPS) representing the ground state of the quantum system after DMRG.
"""
    # Generate an index set representing quantum sites with spin S=1/2
    sites = siteinds("S=1/2", N)

    # Initialize an operator sum representing the Hamiltonian using ITensors
    ampo = OpSum()

    # Construct the Hamiltonian operator for the TFIM
    for j = 1:N
        if j < N
            # Interaction term between neighboring spins (Sz-Sz)
            ampo += -2.0, "Sz", j, "Sz", j + 1
        end
        # Transverse field term (Sx) at each site
        ampo += -h, "Sx", j
    end

    # Convert the operator sum to a Matrix Product Operator (MPO)
    H = MPO(ampo, sites)

    # Initialize the initial state to a random maxM=10 state 
    psi0 = randomMPS(sites,10)

    # Set up parameters for the DMRG (Density Matrix Renormalization Group) algorithm
    sweeps = Sweeps(swp_num)
    setmaxdim!(sweeps, maxM)
    setcutoff!(sweeps, 1E-16)

    # Perform the DMRG simulation to obtain the ground state energy and MPS representation
    energy, psi = dmrg(H, psi0, sweeps)

    return energy, psi
end;

In [7]:
N = 40
h = 0.1
swp_num = 20
maxM = 20
energy, psi = TFIM_DMRG(N, h, swp_num, maxM);

After sweep 1 energy=-19.552487159036072  maxlinkdim=20 maxerr=6.72E-11 time=24.199
After sweep 2 energy=-19.552534465172567  maxlinkdim=16 maxerr=2.54E-14 time=0.084
After sweep 3 energy=-19.552534465211917  maxlinkdim=3 maxerr=9.57E-17 time=0.055
After sweep 4 energy=-19.552534465211927  maxlinkdim=3 maxerr=6.63E-17 time=0.077
After sweep 5 energy=-19.552534465211938  maxlinkdim=3 maxerr=6.63E-17 time=0.046
After sweep 6 energy=-19.552534465211938  maxlinkdim=3 maxerr=6.63E-17 time=0.045
After sweep 7 energy=-19.552534465211938  maxlinkdim=3 maxerr=6.63E-17 time=0.055
After sweep 8 energy=-19.552534465211938  maxlinkdim=3 maxerr=6.63E-17 time=0.056
After sweep 9 energy=-19.552534465211938  maxlinkdim=3 maxerr=6.63E-17 time=0.083
After sweep 10 energy=-19.552534465211938  maxlinkdim=3 maxerr=6.63E-17 time=0.044
After sweep 11 energy=-19.552534465211938  maxlinkdim=3 maxerr=6.63E-17 time=0.048
After sweep 12 energy=-19.552534465211938  maxlinkdim=3 maxerr=6.63E-17 time=0.061
After swee

# 1. J1-J2 spin 1/2 chain

## 1. Heisenberg Hamiltonian construction for finite J2 term

Using

$\vec{S_{i}\cdot}\vec{S}_{j}=S_{i}^{z}S_{j}^{z}+\frac{1}{2}(S_{i}^{+}S_{j}^{-}+S_{i}^{-}S_{j}^{+})$

The hamiltonian gets the form

$ H=J_{1}\sum_{i}\vec{S_{i}\cdot}\vec{S}_{i+1}+J_{2}\sum_{i}\vec{S_{i}}\cdot\vec{S_{i+2}}$

$=J_{1}\sum_{i}\left(S_{i}^{z}S_{i+1}^{z}+\frac{1}{2}(S_{i}^{+}S_{i+1}^{-}+S_{i}^{-}S_{i+1}^{+})\right)+J_{2}\sum_{i}\left(S_{i}^{z}S_{i+2}^{z}+\frac{1}{2}(S_{i}^{+}S_{i+2}^{-}+S_{i}^{-}S_{i+2}^{+})\right) $

Generalization of the method seen in class 6 for the MPO will lead to the matrices


$W_{(L)}=\left(\begin{array}{c}
I\\
S^{z}\\
S^{+}\\
S^{-}\\
0\\
0\\
0\\
0
\end{array}\right)$

$W_{(i)}=\left(\begin{array}{cccccccc}
I & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
S^{z} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
S^{+} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
S^{-} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & I & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & I & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & I & 0 & 0 & 0 & 0\\
0 & J_{1}S^{z} & \frac{J_{1}}{2}S^{-} & \frac{J_{1}}{2}S^{+} & J_{2}S^{z} & \frac{J_{2}}{2}S^{-} & \frac{J_{2}}{2}S^{+} & I
\end{array}\right)$

$W_{(1)}=\left(\begin{array}{cccccccc}
0 & J_{1}S^{z} & \frac{J_{1}}{2}S^{-} & \frac{J_{1}}{2}S^{+} & J_{2}S^{z} & \frac{J_{2}}{2}S^{-} & \frac{J_{2}}{2}S^{+} & I\end{array}\right)$